In [75]:
import pandas as pd
import os 
import duckdb
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
path = os.getcwd() #current path 

In [22]:
# reading into the TTCTscore data
# there are 7 scores columns for each stage(the begining and the ending)
TTCTscore = pd.read_excel(path + "/../data/dataset1_TTCTscore.xlsx")
TTCTscore.head()

,Number,IntendedMajor,Hometown,Race,Gender,AugFluencySA,FlexibilitySA,OriginalitySA,BeginAverageSA,AUGRSFluency,RSFlexibility,RSOriginality,DECFluencySA,FlexibilitySA.1,OriginalitySA.1,EndAverageSA,DECRSFluency,RSFlexibility.1,RSOriginality.1
0,1,zoology,small city,asian,1,100.0,103.0,92.0,98.0,77,44,49,100.0,122.0,101.0,108.0,89,61,73
1,2,biology,small city,asian,2,77.0,77.0,78.0,77.0,55,34,41,92.0,95.0,92.0,93.0,70,41,55
2,3,biology,small city,asian,2,111.0,98.0,106.0,150.0,94,42,65,108.0,111.0,106.0,108.0,110,54,85
3,4,biology,rural area,white,2,92.0,92.0,88.0,91.0,67,39,45,101.0,102.0,93.0,99.0,91,46,57
4,5,biochemistry,large city,black,1,84.0,87.0,92.0,88.0,58,36,48,91.0,93.0,90.0,91.0,69,40,51


In [23]:
# there are some missing score in the TTCTScore dataset, so let us to see it
# We can get the missing number for each variable
TTCTscore.isna().sum()
TTCTscore[TTCTscore['AugFluencySA'].isna() & TTCTscore['DECFluencySA'].isna()]

,Number,IntendedMajor,Hometown,Race,Gender,AugFluencySA,FlexibilitySA,OriginalitySA,BeginAverageSA,AUGRSFluency,RSFlexibility,RSOriginality,DECFluencySA,FlexibilitySA.1,OriginalitySA.1,EndAverageSA,DECRSFluency,RSFlexibility.1,RSOriginality.1
118,119,biochemistry,town,white,2,NaN,NaN,NaN,NaN,66,31,50,NaN,NaN,NaN,NaN,75,39,62
121,122,biochemistry,town,multi.,1,NaN,NaN,NaN,NaN,89,45,65,NaN,NaN,NaN,NaN,113,47,91


In [24]:
# How should we deal with these missing values in TTCT score?
# Could we fill each missing value with a mean of the column averege and the corresponding row average?
# for example if a student's AugFluencySA is missing, we can calculate the average of the AugFluencySA(might be nearly to 100)
#   then we can calculate the teh mean of his/her score in his row(we need to make chooses about what variable will be used to
#   get the mean numebr). Then we get the mean of two averages as the final number to fill this missing data.


In [25]:
# reading into the creativity assessment at the begining
BeginCrAssessment = pd.read_excel(path + "/../data/dataset2_beginingCreativityAssessment.xlsx")
BeginCrAssessment.head()

,Timestamp,FirstName,LastName,ID,Instructor,IntendedMajor,ZipCode,HometownDescription,Gender,Race,...,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
0,2021-08-17 15:17:42.000,Ashton,Johnson,200409390,Dr. Erica Kosal,Nutrition,28412,Small city,Female,Caucasian; White,...,a product is polished through critical thinkin...,can center around an idea or a product,Breadth (recognizing insights in more than one...,Are well organized and consider projects in a ...,all of the above,Hypotheses and theories are creative ideas,"No, but it may show critical thinking","Yes, this shows solid creative thinking","No, but it may show solid critical thinking","Yes, this shows solid creative thinking"
1,2021-08-17 19:55:34.932,Lilly,Wallace,200178656,Dr. Kenny Kuo,Genetics,27616,Suburb,Female,Caucasian; White,...,"during the preparation stage, you make connect...",can center around an idea or a product,Breadth (recognizing insights in more than one...,"Start a project, and then stop and tend to thi...",all of the above,Hypotheses and theories are creative ideas,"No, but it may show critical thinking","Yes, this shows solid creative thinking","Maybe there is creative thinking, but I need m...","Yes, this shows solid creative thinking"
2,2021-08-19 09:47:35.011,Mason,Hock,200343314,Dr. Erica Kosal,Biochemistry,27403,Small city,Male,Caucasian; White,...,a product is polished through critical thinkin...,can center around an idea or a product,Adhering to the scientific method,"Start a project, and then stop and tend to thi...",all of the above,All of the above,"Yes, this shows solid creative thinking","Maybe there is creative thinking, but I need m...","Maybe there is creative thinking, but I need m...",There is evidence of both creative and critica...
3,2021-08-18 20:32:01.619,Alexandra,Kenna,200347101,Dr. Kenny Kuo,Nutrition,60126,Suburb,Female,Caucasian; White,...,"during the preparation stage, you make connect...",can center around an idea or a product,Breadth (recognizing insights in more than one...,"Start a project, and then stop and tend to thi...",all of the above,Hypotheses and theories are creative ideas,"No, but it may show critical thinking","Yes, this shows solid creative thinking","No, but it may show solid critical thinking",There is evidence of both creative and critica...
4,2021-08-18 00:11:46.034,Logan,Braun,200350268,Dr. Kenny Kuo,Biology - BA,27713,Suburb,Female,Caucasian; White,...,during the incubation stage you gather a lot o...,can center around an idea or a product,Fairness (treating all sides alike without ref...,Tend to work on one project at a time and perf...,connecting and combining things that aren’t ob...,Such creativity is “free” from the constraints...,"No, but it may show critical thinking","Yes, this shows solid creative thinking","No, but it may show solid critical thinking","Yes, this shows solid creative thinking"


In [26]:
# reading the creativity assessment at the begining
EndCrAssessment = pd.read_excel(path + "/../data/dataset3_endingassessment.xlsx")
EndCrAssessment.head()

,Timestamp,Score(outof15),LastName,StudentIDnumber,Instructor,IntendedMajor,ZipCode,HometownDescription,Gender,Race,...,Q7,Q8,Q9,Q10,Q11,F1,F2,F3,F4,F5
0,2021-11-02 15:15:16.206,12,Adam Malik,200398445,Dr. Erica Kosal,Zoology,27502,Small city,Male,Asian,...,Such creativity is “free” from the constraints...,"No, but it may show critical thinking","Yes, this shows solid creative thinking","No, but it may show solid critical thinking","Yes, this shows solid creative thinking",True,False,hanging out with the same intelligent people a...,shows a goodness of fit between the problem po...,play allows you to image possibilities
1,2021-11-02 15:17:00.000,8,Angeles Solano,200400013,Dr. Erica Kosal,Biology - BS,28726,Town,Female,Hispanic; Latinx,...,Hypotheses and theories are creative ideas,"No, but it may show critical thinking","Maybe there is creative thinking, but I need m...","No, but it may show solid critical thinking",There is evidence of both creative and critica...,True,False,hanging out with the same intelligent people a...,shows a goodness of fit between the problem po...,play allows you to interact with other people
2,2021-10-28 10:01:51.232,11,Barker,200405063,Dr. Erica Kosal,A major outside of science,28719,Rural Area,Female,Caucasian; White,...,Hypotheses and theories are creative ideas,"Yes, this shows solid creative thinking","Yes, this shows solid creative thinking","No, but it may show solid critical thinking","No, there is no evidence creativity is used here",True,False,"having an attitude of fixing things, connectin...",is of value to the discipline,play allows you to image possibilities
3,2021-11-02 09:54:19.862,13,Barnhill,200405077,Dr. Erica Kosal,Biology - BS,27609,Large city,Female,Caucasian; White,...,Hypotheses and theories are creative ideas,"No, but it may show critical thinking","Yes, this shows solid creative thinking","No, but it may show solid critical thinking","Yes, this shows solid creative thinking",False,False,hanging out with the same intelligent people a...,shows a goodness of fit between the problem po...,play allows you to image possibilities
4,2021-11-02 15:12:45.242,9,Basnight,200405114,Dr. Erica Kosal,Biology - BS,27953,Town,Female,Caucasian; White,...,Hypotheses and theories are creative ideas,"No, but it may show critical thinking","Yes, this shows solid creative thinking","Maybe there is creative thinking, but I need m...",There is evidence of both creative and critica...,False,True,hanging out with the same intelligent people a...,shows a goodness of fit between the problem po...,play allows you to image possibilities


In [27]:
# rename the ID column name
EndCrAssessment.rename({'StudentIDnumber':'ID'}, inplace=True, axis=1)
EndCrAssessment.shape

(265, 39)

In [28]:
# reading the highschool gpa data
HSGpa = pd.read_excel(path + "/../data/dataset4_highschoolGPA.xlsx")
HSGpa.head()

,Student ID,Deidentified Number,unweighted_hs_gpa,weighted_hs_gpa
0,200398445,1,3.637,4.086
1,200404629,2,3.882,4.500
2,200404663,3,3.836,4.344
3,200394288,4,4.000,4.540
4,200430376,5,3.774,4.359


In [29]:
# renaming the column name
HSGpa.rename({'Student ID': 'ID'}, inplace=True, axis=1)

In [30]:
# get the student rows are in the study
HSG_inStudy = HSGpa[HSGpa['Deidentified Number'] != 'XX']
HSG_inStudy.shape

(243, 4)

In [31]:
# using sql to combine gpa data and BeginCrAssessment into one dataset by Student ID
BeginCrAssessmentandHSGpa = duckdb.query("select * from BeginCrAssessment as b inner join HSGpa as h on b.ID == h.ID").to_df()
BeginCrAssessmentandHSGpa.shape

(277, 39)

In [32]:
# have a look the missing gpa in this dataset
BeginCrAssessmentandHSGpa.isna().sum()[-2:]
# TTCTscore[TTCTscore['AugFluencySA'].isna() & TTCTscore['DECFluencySA'].isna()]

unweighted_hs_gpa    7
weighted_hs_gpa      7
dtype: int64

In [33]:
# using sql to combine gpa data and BeginCrAssessment into one dataset by Student ID
EndCrAssessmentandHSGpa = duckdb.query("select * from EndCrAssessment as b inner join HSGpa as h on b.ID == h.ID").to_df()
EndCrAssessmentandHSGpa.shape

(264, 43)

In [34]:
# let see how many missing gpa for the CrAssessment data 
EndCrAssessmentandHSGpa.isna().sum()[-2:]

unweighted_hs_gpa    3
weighted_hs_gpa      3
dtype: int64

In [52]:
# t-test for two sample
# Get two TTCTscore data for begin and end
# get beginAverageSA and drop the nan
avg_begin_ttct = TTCTscore.BeginAverageSA.dropna().values


In [53]:
# get endAverageSA and drop the nan
avg_end_ttct = TTCTscore.EndAverageSA.dropna().values

In [55]:
len(avg_begin_ttct)

238

In [58]:
# Print the variance of both data groups
print(np.var(avg_begin_ttct), np.var(avg_end_ttct))
#Here, the ratio is 210/121 which is less than 4:1. 

210.84330202669307 121.97712757771623


In [59]:
# Perform the two sample t-test with equal variances
stats.ttest_ind(a=avg_begin_ttct, b=avg_end_ttct, equal_var=True)

Ttest_indResult(statistic=-2.620481906630271, pvalue=0.00906855508795876)

In [87]:
# drop missing value
avg_ttct = TTCTscore[['BeginAverageSA','EndAverageSA']].dropna().reset_index(drop=True)
# paired t-test 
stats.ttest_rel(avg_ttct.iloc[:, 0], avg_ttct.iloc[:, 1])

Ttest_relResult(statistic=-3.720913916210963, pvalue=0.0002526024795969336)

In [85]:
# the p value is less than 0.05. There is enough evidence to say that values of begin and end score are different.
# Which tell there are changes happened after students enrolling the class.

,BeginAverageSA,EndAverageSA
0,98.0,108.0
1,77.0,93.0
2,150.0,108.0
3,91.0,99.0
4,88.0,91.0
...,...,...
243,95.0,89.0
244,76.0,90.0
245,79.0,93.0
246,74.0,92.0
